In [23]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
%config Completer.use_jedi = False
import utils.db as db
import glob
import datetime
import folia.main as folia
from utils import db
mongo = db.get_db()
import re
import regex

1. Download a fragment
2. Check if USC
3. If so get all transcripts
4. Get a word sequence from it
5. Find it in the transcripts

In [2]:
def get_all_fragments():
    mongo = db.get_db()
    results = mongo.fragments.find({},{'_id':0})
    results = [element for element in results]
    all_mid_nodes =[]
    all_fragments = []
    for result in results:
        mid_nodes = [element for element in result['tree']['children'] if len(element['children'])>0]
        for mid_node in mid_nodes:

            all_mid_nodes.append(mid_node['label'])
            for fragment in mid_node['children']:
                all_fragments.append({'fragment':fragment,'mid_label':mid_node['label'],'top_label':result['tree']['label']})
            
    return (all_fragments)

In [3]:
def update_fragments(main_node_label,mid_node_label,fragment,attributes):
    results = mongo.fragments.find({'label':main_node_label},{'_id':0})[0]
    mongo_id= mongo.fragments.find({'label':main_node_label},{'_id':1})[0]
    for x,element in enumerate(results['tree']['children']):
        if element['label']==mid_node_label:
            for n,leaves in enumerate(element['children']):
                if leaves['label'] == fragment['label']:
                    for attr in attributes:
                        results['tree']['children'][x]['children'][n][list(attr.keys())[0]]=attr[list(attr.keys())[0]]
                    
    replace = mongo.fragments.replace_one(mongo_id,results)


In [4]:
def update_mid_node_label(main_node_label,mid_node_orig_label,mid_node_new_label):
    results = mongo.fragments.find({'label':main_node_label},{'_id':0})[0]
    mongo_id= mongo.fragments.find({'label':main_node_label},{'_id':1})[0]
    for x,element in enumerate(results['tree']['children']):
        if element['label']==mid_node_orig_label:
            results['tree']['children'][x]['label']=mid_node_new_label
    replace = mongo.fragments.replace_one(mongo_id,results)
            
            

In [5]:
def delete_mid_node(main_node_label,mid_node_orig_label):
    results = mongo.fragments.find({'label':main_node_label},{'_id':0})[0]
    mongo_id= mongo.fragments.find({'label':main_node_label},{'_id':1})[0]
    to_delete=[]
    for x,element in enumerate(results['tree']['children']):
        if element['label']==mid_node_orig_label:
            to_delete.append(x)
            break
    for element in to_delete:
        del results['tree']['children'][element]
    replace = mongo.fragments.replace_one(mongo_id,results)
            
            

In [6]:
def eliminate_double_parenthesis(all_fragments):
    for fragment in all_fragments:
        if '(..) (..)' == fragment['fragment']['label'][0:9]:
            label = fragment['fragment']['label'][5:]
            update_fragments(fragment['top_label'],fragment['mid_label'],fragment['fragment'],[{'label':label}])   

In [7]:
def correct_negation_trad_quote(all_fragments):
    for fragment in all_fragments:
        if len(fragment['fragment']['label'].split(" n't"))==2:
            split = fragment['fragment']['label'].split(" n't")
            label = split[0]+"n't"+split[1]
            update_fragments(fragment['top_label'],fragment['mid_label'],fragment['fragment'],[{'label':label}]) 
        if len(fragment['fragment']['label'].split(" n't"))==3:
            split = fragment['fragment']['label'].split(" n't")
            label = split[0]+"n't"+split[1]+"n't"+split[2]
            update_fragments(fragment['top_label'],fragment['mid_label'],fragment['fragment'],[{'label':label}]) 
        if len(fragment['fragment']['label'].split(" n't"))==4:
            split = fragment['fragment']['label'].split(" n't")
            label = split[0]+"n't"+split[1]+"n't"+split[2]+"n't"+split[3]
            update_fragments(fragment['top_label'],fragment['mid_label'],fragment['fragment'],[{'label':label}]) 

In [8]:
def correct_negation_new_quote(all_fragments):
    for fragment in all_fragments:
        if len(fragment['fragment']['label'].split(" n’t"))==2:
            split = fragment['fragment']['label'].split(" n’t")
            label = split[0]+"n't"+split[1]
            update_fragments(fragment['top_label'],fragment['mid_label'],fragment['fragment'],[{'label':label}]) 
        if len(fragment['fragment']['label'].split(" n’t"))==3:
            split = fragment['fragment']['label'].split(" n’t")
            label = split[0]+"n't"+split[1]+"n't"+split[2]
            update_fragments(fragment['top_label'],fragment['mid_label'],fragment['fragment'],[{'label':label}]) 
        if len(fragment['fragment']['label'].split(" n’t"))==4:
            split = fragment['fragment']['label'].split(" n’t")
            label = split[0]+"n't"+split[1]+"n't"+split[2]+"n’t"+split[3]
            update_fragments(fragment['top_label'],fragment['mid_label'],fragment['fragment'],[{'label':label}]) 

In [15]:
def delete_fragment(all_fragments,label):
    res = []
    for fragment in all_fragments:
        if fragment['fragment']['label'] == label:
            main_node_label = fragment['top_label']
            mid_node_label = fragment['mid_label']
            results = mongo.fragments.find({'label':main_node_label},{'_id':0})[0]
            mongo_id= mongo.fragments.find({'label':main_node_label},{'_id':1})[0]
            for x,element in enumerate(results['tree']['children']):
                if element['label']==mid_node_label:
                    for n,leaves in enumerate(element['children']):
                        if leaves['label'] == label:
                            res.append((x,n))
                            
    for element in res[0:1]:
        del results['tree']['children'][element[0]]['children'][element[1]]
    if len(res)>0:
        replace = mongo.fragments.replace_one(mongo_id,results)        

In [10]:
all_fragments = get_all_fragments()

In [16]:
label = "I just slept there the rest of the night, but he was dead just from cold and not eating."
delete_fragment(all_fragments, label)
label = "(..) they didn't care, or they hit your head or your leg, or they hit a child."
delete_fragment(all_fragments, label)

In [17]:
delete_mid_node('shaking','together')
delete_mid_node('shame','horrible')
delete_mid_node('run','scream')

In [ ]:
len(fragment['fragment']['label'].split(" n’t"))

In [18]:
correct_negation_new_quote(all_fragments)

In [20]:
correct_negation_trad_quote(all_fragments)

In [21]:
eliminate_double_parenthesis(all_fragments)

In [ ]:
update_mid_node_label('cry','','God')

In [ ]:
#update_fragments("rape","kill",fragment['fragment'],[{'media_index':100}])

In [26]:
new_labels = []
for fragment in all_fragments:
    testimony_id = fragment['fragment']['testimony_id']
    label = fragment['fragment']['label']
    #label = "we were standing completely in the nude and... I was frightened. I was thirteen years old (..)."
    
    
    #pattern_parenthesis_with_no_space = r'(?<!(\s|\n))(\(\.{1,3}\))(?!(\s|\.))'
    
    pattern_parenthesis_with_no_space_before = r'(\w)(\(\.{1,3}\))'
    regex_parenthesis_with_no_space_before = re.compile(pattern_parenthesis_with_no_space_before, re.IGNORECASE)
    #print (regex_parenthesis_with_no_space_before.search(label))
    label = re.sub(pattern_parenthesis_with_no_space_before, r"\1 \2", label)
    #print (result)

    pattern_parenthesis_with_no_space_after = r'(\(\.{1,3}\))(\w)'
    regex_parenthesis_with_no_space_after = re.compile(pattern_parenthesis_with_no_space_after, re.IGNORECASE)
    #print (regex_parenthesis_with_no_space_after.search(label))
    label = re.sub(pattern_parenthesis_with_no_space_after, r"\1 \2", label)
    
    
    
    
    #pattern_dots = r'(^\.{0,3})(\w)'
    #regex_dots = re.compile(pattern_dots, re.IGNORECASE)
    #label = regex_dots.sub(r"(..) \2",label)
    
    
    #label = regex.sub(' (..) ', label)
        
    label = re.sub(' +', ' ',label)
    label = label.strip()
    
    
    
    
    # Check if ending with character
    
    if (label[-1].isalpha()) or label[-1] ==')':
   
        label = label+"."
    if (label[0].islower()):
        label='(..) '+label
        
    if label[-1]=='…':
        label = label[:-1]+' (..).'
        
        
    if label [0:3]=='..,':
        label = '(..) ' +label[3:]
    
    if label [0:2]=='..':
        label = '(..) ' +label[2:]
        
    if label[0]=='.':
        label = '(..) ' +label[1:]
        

    
    
    
    #label = regex.sub(' (..) ', label)
        
    label = re.sub(' +', ' ',label)
    label = label.strip()
    label = label.strip()  
    
    new_labels.append(label)
    update_fragments(fragment['top_label'],fragment['mid_label'],fragment['fragment'],[{'label':label}])      

The survival , the guilt of survivors - I don't have guilt for surviving.
But I didn't only shiver from the cold but from the fear of what is ahead of me (..).
(..) terrible screaming, all sweat and shaking. I couldn't -- I could never go back to sleep (..).
I couldn't swallow food. I was shaking inside.
I got a sort of nervous breakdown. And I started shaking. I shook so violently that I couldn't talk.
I was shaking like that. I really couldn't put a straight step (..).
(..) she wiped her hand and shivered. She never touched it, never touched her soup. She couldn't drink (..).
Blood was all over. And, yeah, after, I was shaking. I couldn't (..).
And we stayed there, and everybody was shaking like mad. We didn't have nothing to eat (..).
But we didn't eat. We didn't have no food. And my lips were black, and my teeth were loose. I remember shaking my teeth.
(..) killing a human being really isn't the biggest crime. It 's to degrade him, to -- to dehumanize him (..).
I wasn't painful as 

In [ ]:
# -> x I just slept there the rest of the night, but he was dead just from cold and not eating.
# -> x they didn't care, or they hit your head or your leg, or they hit a child. It was merciless.
# -> x (..) (..) they told me to be very quiet and not to move and I remember feeling that, that I wa suffocating and I couldn't breathe (..). x
# -> x (..) we were all sort of sitting together and -- and, you know, trembling.
# stand and shiver?  -> USHMM
#  -> x Cry and God 
# -> x shame  -> horrible
# -> was a shameful existence, you could n't do anything about it.
# I walked away from other people that were killed behind me who couldn't walk all day long.
# ->the older people could n’t run as fast
# People running. People screaming. -> USHMM

In [204]:
end = list(set([element[-1] for element in new_labels]))

count(2)

In [198]:
end = list(set([element for element in new_labels if element[-1]=='…']))

In [202]:
end[0][:-1]

'I started shaking and shivering so bad. It was terrible. There was Mengele with two dogs and a German woman'

In [159]:
start[1][0:3]=='..,'

False

In [146]:
start

['..we didnt go naked anywhere, and I was so embarrassed being naked, and I tried to cover myself up.',
 '..,we lived in tremendous fear (..).',
 '.. you have tremendous guilt thinking about it, and would have the rest of your life.',
 ".(..) they told me to be very quiet and not to move and I remember feeling that, that I wa suffocating and I couldn't breathe (..).",
 '..oh God, again, naked (..) we were shy, so shy and so ashamed for the man to disinfect you.']

In [127]:
pattern_dots = r'^(\.{0,3})(.*)'
regex_dots = re.compile(pattern_dots, re.IGNORECASE)
regex_dots.sub(r"(..) \2",start[0])

'(..) we didnt go naked anywhere, and I was so embarrassed being naked, and I tried to cover myself up.'

In [101]:
start[0]

IndexError: list index out of range

In [67]:
label

'One day, I was sitting after work in the bunk there, in the barrack, and I collapsed.'